In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
pd.set_option('display.max_rows', None)
import zipfile
import os
import shutil

In [2]:
scenario_name = "hack fixed, twm, fixed long trips"
path_to_zip_file = '/home/ai6644/Malmo/Tools/Simulation results/Lolland/tw around departure, stretched, with transfers/'+\
                   scenario_name+'/log.zip'

In [3]:
day_sec = int(86400)
day_h = int(24)

In [4]:
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    try:
        shutil.rmtree('data/zip')
    except (FileNotFoundError, OSError) as e:
        pass
    os.mkdir('data/zip')
    zip_ref.extractall('data/zip')
path = 'data/zip/' + os.listdir('data/zip')[0] + '/vehicle_logs'

In [5]:
# all_files = glob.glob(path + "/vehicle_occupancy_*")
all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_occupancy_*")
all_files.sort()

occupancies = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['time'] = df.time.astype(int)
    df = df.drop_duplicates(keep='last', subset='time')
    occupancies.append(df)

merged_time = pd.concat(occupancies)['time'].sort_values().drop_duplicates()

# for df in occupancies:
#     df['time'] = df.time.astype(int).drop_duplicates()
#     df.set_index('time', inplace=True)
    
# merged_df = pd.concat(occupancies, axis=1).fillna(method='ffill')

In [6]:
n_cars = len(all_files)
# n_cars = 25

In [7]:
all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_status_*")
all_files.sort()

statuses = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    df.columns = ['time', 'status']
    df['time'] = df.time.astype(int)
    df = df.drop_duplicates(keep='last', subset='time')
    statuses.append(df)

In [8]:
occupancies[1]

,time,#passengers
0,0,0
1,18212,0
2,18237,0
3,18467,0
4,18512,0
5,18785,0
6,18905,0
8,18948,0
10,19776,0
11,19850,1


In [9]:
times = []
for vid, status in enumerate(statuses):
    t_idle = 0
    t_wait = 0
    t_drive = 0
    t_return = 0
    t_stop = 0
    started = False # not used, but we may use it to add empty waiting time to idle time
    for (_, st1), (_,st2) in zip(status.iterrows(), status.iloc[1:].iterrows()):

        t = st2.time - st1.time

        # departed
        if st1.status == 6 and st2.status != 6:
            started = True
        # staying at depot
        if st1.status == 6:
            t_idle += t
        # arrived to depot
        if st1.status != 6 and st2.status == 6:
            started = False

        if st1.status == 4:
            t_wait += t

        # driving
        if st1.status == 3:
            t_drive += t

        # returning
        if st1.status == 5:
            t_return += t

        #on a stop
        if st1.status in (0,2):
            t_stop +=t

    if len(status) == 1:
        t_idle = 86400
    else:
        t_idle += 86400 - st2.time

    if (t_idle + t_wait + t_drive + t_return + t_stop != 86400):
        print('somehing is wrong!')
        break

    times.append(pd.DataFrame([t_idle, t_wait, t_drive, t_return, t_stop], index=['idle', 'wait', 'drive', 'return', 'stop'], columns=[vid]).T)

In [10]:
vehicle_times = pd.concat(times)
vehicle_times

,idle,wait,drive,return,stop
0,56537,3018,21272,653,4920
1,29039,1001,45364,797,10199
2,53825,251,26105,99,6120
3,43750,1736,32866,8,8040
4,63904,1873,15646,1617,3360
5,66274,2010,14491,25,3600
6,74398,60,10127,15,1800
7,59616,517,21174,653,4440
8,76155,807,7635,3,1800
9,48047,1016,29727,1012,6598


In [11]:
vehicle_times = vehicle_times[vehicle_times.idle != 86400]
vehicle_times

,idle,wait,drive,return,stop
0,56537,3018,21272,653,4920
1,29039,1001,45364,797,10199
2,53825,251,26105,99,6120
3,43750,1736,32866,8,8040
4,63904,1873,15646,1617,3360
5,66274,2010,14491,25,3600
6,74398,60,10127,15,1800
7,59616,517,21174,653,4440
8,76155,807,7635,3,1800
9,48047,1016,29727,1012,6598


In [12]:
len(vehicle_times)

29

In [13]:
vehicle_times.sum()

idle      1611897
wait        36987
drive      697084
return      10958
stop       148674
dtype: int64

# Cost for DRT (assuming availability between 5 and 20 - 9h)

In [14]:
(vehicle_times.drive + vehicle_times.stop + vehicle_times['return']).sum()*(367/60/60) + \
    ((vehicle_times.idle + vehicle_times.wait).sum() - len(vehicle_times)*7*60*60)*(367/60/60)*0.7

152852.93099999998

In [15]:
# waiting time is full prised
(vehicle_times.drive + vehicle_times.stop + vehicle_times['return'] + vehicle_times.wait).sum()*(367/60/60) + \
    ((vehicle_times.idle).sum() - len(vehicle_times)*9*60*60)*(367/60/60)*0.7

139083.91674999997

In [16]:
cost = 440
(vehicle_times.drive + vehicle_times.stop + vehicle_times['return'] + vehicle_times.wait).sum()*(cost/60/60) + \
    ((vehicle_times.idle).sum() - len(vehicle_times)*9*60*60)*(cost/60/60)*0.7

166749.11

In [17]:
cost = 800
(vehicle_times.drive + vehicle_times.stop + vehicle_times['return'] + vehicle_times.wait).sum()*(cost/60/60)

198600.6666666667

# Cost for busses

In [18]:
REQUIRES UPDATING
# +1h to ride to depot
# 715: 2 buses from 5.50-19.20 and 6.15-19.20 with 4+5 breaks of 30 minutes and two of an hour
l715 = 1200*(19.3-5.8+1 + 19-6+1 -9*0.5-2) + 1200*0.7*(9*0.5 + 2)
# 716: 2 buses from 5.40-17.50 and 6.20-18.50 (with 4 break of 50 minutes) and one more 6.30-8.50 + 18.48-19.50
l716 = 1200*(17.8-5.66+1 + 18.83-6.33+1 - 2*4) + 1200*0.7*4*2 + 1200*(3.3+2)
# 718: 1 bus full time 5.43-18.09
l718 = 1200*(18.16-5.72 + 1)
# 778: 1 bus 14.15-16.30 + 6.55-7.45
l788 = 1200*(2+1 + 1+1)
# 791: 1 bus 14-17 + 7-8
l791 = 1200*(17-14 + 1 + 1+1)
# line 774: 1 bus 6.25-7.55 + 13.00+16.15
l774 = 1200*(1.3+1 + 3.2+1)
# line 772: 1 bus 6.35-7.35 + 13.45-16.35
l772 = 1200*(1+1 + 3+1)
# line 792: 1 bus 7-7.35 + 14.20-17.20
l792 = 1200*(0.5+1 + 3+1)
# line 780: 2 buses 6-20 with 3.40 hours of a break
l780 = 1200*((20-6 + 1)*2 - 3.6) + 1200*0.7*3.6
# line 719: 2 busses 6-19 and 7-19 7.40 pause + 1 trip 7.10-7.40
l719 = 1200*(19-6+1 + 19-7+1 -7.6 +0.5+1) + 1200*0.7*7.6
# line 711: one bus 6.50-17.44
l711 = 1200*(17-6 + 1)
# line 714: 1 bus 7.50-17.13 with 9*0.37 pause
l714 = 1200*(10.2 - 9*0.55 + 1) + 1200*0.7*9*0.55
# line 713: 1 bus 7.20-16.40 with 9*0.40 pause
l713 = 1200*(9.3 - 9*0.6 + 1) + 1200*0.7*9*0.6
# line 771: 6.30-7.30 + 13.45-16.35 (2x0.50 pause)
l771 = 1200*(1+1 + 2.8+1 - 1.6) + 1200*0.7*1.6 
# line 773: 6.40-7.32 + 13.40-16.30 (2x0.50 pause)
l773 = 1200*(0.85+1 + 2.8+1 - 1.6) + 1200*0.7*1.6
# line 778: 6.55-7.45 + 14.15-16.30
l778 = 1200*(0.83+1 + 2.25 + 1)

SyntaxError: invalid syntax (<ipython-input-18-540e15537d6b>, line 1)

In [19]:
# No extra time added
# 715: 2 buses from 5.50-19.20 and 6.15-19.20 with 4+5 breaks of 30 minutes and two of an hour
l715 = 1200*(19.3-5.8 + 19-6 -9*0.5-2) + 1200*0.7*(9*0.5 + 2)
# 716: 2 buses from 5.40-17.50 and 6.20-18.50 (with 4 break of 50 minutes) and one more 6.30-8.50 + 18.48-19.50
l716 = 1200*(17.8-5.66 + 18.83-6.33 - 2*4) + 1200*0.7*4*2 + 1200*(3.3+2)
# 718: 1 bus full time 5.43-18.09
l718 = 1200*(18.16-5.72)
# 778: 1 bus 14.15-16.30 + 6.55-7.45
l788 = 1200*(2 + 1)
# 791: 1 bus 14-17 + 7-8
l791 = 1200*(17-14  + 1)
# line 774: 1 bus 6.25-7.55 + 13.00+16.15
l774 = 1200*(1.3 + 3.2)
# line 772: 1 bus 6.35-7.35 + 13.45-16.35
l772 = 1200*(1 + 3)
# line 792: 1 bus 7-7.35 + 14.20-17.20
l792 = 1200*(0.5 + 3)
# line 780: 2 buses 6-20 with 3.40 hours of a break
l780 = 1200*((20-6)*2 - 3.6) + 1200*0.7*3.6
# line 719: 2 busses 6-19 and 7-19 7.40 pause + 1 trip 7.10-7.40
l719 = 1200*(19-6 + 19-7 -7.6 +0.5) + 1200*0.7*7.6
# line 711: one bus 6.50-17.44
l711 = 1200*(17-6)
# line 714: 1 bus 7.50-17.13 with 9*0.37 pause
l714 = 1200*(10.2 - 9*0.55) + 1200*0.7*9*0.55
# line 713: 1 bus 7.20-16.40 with 9*0.40 pause
l713 = 1200*(9.3 - 9*0.6 ) + 1200*0.7*9*0.6
# line 771: 6.30-7.30 + 13.45-16.35 (2x0.50 pause)
l771 = 1200*(1 + 2.8 - 1.6) + 1200*0.7*1.6 
# line 773: 6.40-7.32 + 13.40-16.30 (2x0.50 pause)
l773 = 1200*(0.85 + 2.8 - 1.6) + 1200*0.7*1.6
# line 778: 6.55-7.45 + 14.15-16.30
l778 = 1200*(0.83 + 2.25)
# line 717: 1 bus 6-9 + 13-16
l717 = 1200*(3 + 3) + 1200*(4*0.7)

In [20]:
l719 + l711 + l713 + l714 + l715 + l716 + l718 + l771 + l772 + l773 + l774 + l778 + l780 + l791 + l792

201162.0

In [21]:
# no 771 and 772 and 780
l719 + l711 + l713 + l714 + l715 + l716*0.5 + l718 + l773 + l774 + l778 + l791 + l792 + l780*0.5 + l717*0.5

164982.0

## Service hours

In [47]:

# 715: 2 buses from 5.50-19.20 and 6.15-19.20 with 4+5 breaks of 30 minutes and two of an hour
l715 = (19.3-5.8 + 19-6 -9*0.5-2) + (9*0.5 + 2)
# 716: 2 buses from 5.40-17.50 and 6.20-18.50 (with 4 break of 50 minutes) and one more 6.30-8.50 + 18.48-19.50
l716 = (17.8-5.66 + 18.83-6.33 - 2*4) + 4*2 + (3.3+2)
# 718: 1 bus full time 5.43-18.09
l718 = (18.16-5.72)
# 778: 1 bus 14.15-16.30 + 6.55-7.45
l788 = (2 + 1)
# 791: 1 bus 14-17 + 7-8
l791 = (17-14  + 1)
# line 774: 1 bus 6.25-7.55 + 13.00+16.15
l774 = (1.3 + 3.2)
# line 772: 1 bus 6.35-7.35 + 13.45-16.35
l772 = (1 + 3)
# line 792: 1 bus 7-7.35 + 14.20-17.20
l792 = (0.5 + 3)
# line 780: 2 buses 6-20 with 3.40 hours of a break
l780 = ((20-6)*2 - 3.6) + 3.6
# line 719: 2 busses 6-19 and 7-19 7.40 pause + 1 trip 7.10-7.40
l719 = (19-6 + 19-7 -7.6 +0.5) + 7.6
# line 711: one bus 6.50-17.44
l711 = (17-6)
# line 714: 1 bus 7.50-17.13 with 9*0.37 pause
l714 = (10.2 - 9*0.55) + 9*0.55
# line 713: 1 bus 7.20-16.40 with 9*0.40 pause
l713 = (9.3 - 9*0.6 ) + 9*0.6
# line 771: 6.30-7.30 + 13.45-16.35 (2x0.50 pause)
l771 = (1 + 2.8 - 1.6) + 1.6 
# line 773: 6.40-7.32 + 13.40-16.30 (2x0.50 pause)
l773 = (0.85 + 2.8 - 1.6) + 1.6
# line 778: 6.55-7.45 + 14.15-16.30
l778 = (0.83 + 2.25)
# line 717: 1 bus 6-9 + 13-16
l717 = (3 + 3)

In [48]:
l719 + l711 + l713 + l714 + l715 + l716 + l718 + l773 + l774 + l778 + l791 + l792 + l780 + l717

177.61